In [1]:
import os
import numpy as np
from torchsummary import summary
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
feature= "raw"
sample_len = 500
num_epochs=99
batch_s= 128

from torch.optim import Adam, AdamW, SGD, RMSprop
from torch.optim.lr_scheduler import MultiStepLR
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning import losses, miners, samplers, testers, trainers
import torch.nn as nn
import torch.nn.functional as F
from pytorch_metric_learning import losses, regularizers
from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics.pairwise import cosine_distances as cd
from collections import defaultdict
from pyeer.eer_info import get_eer_stats
from sklearn.metrics.pairwise import manhattan_distances as md

from sklearn.metrics.pairwise import cosine_similarity as cs

from pytorch_metric_learning.distances import LpDistance, CosineSimilarity,SNRDistance


import h5py
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, Subset
class EEGDataset(Dataset):
    def __init__(self, h5_file_path):
        # Open the HDF5 file
        self.h5_file = h5py.File(h5_file_path, 'r')

        # Access the data and labels from the HDF5 file
        self.x_data = self.h5_file['data']  # EEG data (use memory-mapped access)
        self.y_data = self.h5_file['labels'][:]  # Labels (load fully into memory)
        self.s_data = self.h5_file['sessions'][:]  # Sessions (load fully into memory)

        # Create a mapping from original subject labels to new integer labels (starting from 1)
        unique_subjects = np.unique(self.y_data)  # Get unique subject labels
        self.subject_map = {subject: idx + 1 for idx, subject in enumerate(unique_subjects)}  # Map to range [1, len(unique_subjects)]

        # Map the y_data labels using the subject_map
        self.y_data_mapped = np.array([self.subject_map[subject] for subject in self.y_data])

        # Convert labels to torch tensors
        self.targets = torch.tensor(self.y_data_mapped, dtype=torch.long)

        # Print the number of unique subjects (labels)
        print(f'Number of unique subjects: {len(unique_subjects)}')

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x_tensor = torch.tensor(self.x_data[idx], dtype=torch.float32)  # Convert to torch tensor
        y_tensor = self.targets[idx]  # Fetch the preloaded label tensor
        session = self.s_data[idx]  # Session info

        return x_tensor, y_tensor, session

    def close(self):
        self.h5_file.close()


class EEGDataset(Dataset):
    def __init__(self, h5_file_path):
        # Open the HDF5 file
        with h5py.File(h5_file_path, 'r') as h5_file:
            # Load the data, labels, and sessions fully into memory
            self.x_data = np.array(h5_file['data'])  # Load EEG data into memory
            self.y_data = np.array(h5_file['labels'])  # Load labels into memory
            self.s_data = np.array(h5_file['sessions'])  # Load sessions into memory

        # Create a mapping from original subject labels to new integer labels (starting from 1)
        unique_subjects = np.unique(self.y_data)  # Get unique subject labels
        self.subject_map = {subject: idx + 1 for idx, subject in enumerate(unique_subjects)}  # Map to range [1, len(unique_subjects)]

        # Map the y_data labels using the subject_map
        self.y_data_mapped = np.array([self.subject_map[subject] for subject in self.y_data])

        # Convert labels to torch tensors
        self.targets = torch.tensor(self.y_data_mapped, dtype=torch.long)

        # Print the number of unique subjects (labels)
        print(f'Number of unique subjects: {len(unique_subjects)}')

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        # Convert data and labels into torch tensors
        x_tensor = torch.tensor(self.x_data[idx], dtype=torch.float32)  # Fetch EEG data
        y_tensor = self.targets[idx]  # Fetch the preloaded label tensor
        session = self.s_data[idx]  # Fetch the session info

        return x_tensor, y_tensor, session


# Usage example
h5_file_path_t = f'../Data/train_{feature}.h5'
h5_file_path_v = f'../Data/valid_{feature}.h5'
train_dataset = EEGDataset(h5_file_path_t)
valid_dataset = EEGDataset(h5_file_path_v)
#valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=True)


import torch
import numpy as np

# Example data
subject_ids = train_dataset.targets  # Replace with train_dataset.targets
session_ids = train_dataset.s_data          # Replace with train_dataset.s_data

# Combine subject and session IDs into unique pairs
pairs = list(zip(subject_ids.tolist(), session_ids.tolist()))

# Create a mapping from pairs to unique IDs
unique_pairs = list(set(pairs))  # Get unique pairs
pair_to_id = {pair: idx for idx, pair in enumerate(unique_pairs)}

# Map each pair to its unique ID
unique_ids = torch.tensor([pair_to_id[pair] for pair in pairs])

import torch
import torch.nn as nn

def validate(model, loader, loss_func, mining_func, device):
    model.eval()
    total_loss = 0.0
    total_batches = 0

    with torch.no_grad():
        # Iterate over the validation loader (using the provided loader, not train_loader)
        for batch_idx, (data, labels, sessions) in enumerate(loader):
            # Move data and labels to the specified device
            inputs = data.to(device)
            targets = labels.to(device)

            # Compute embeddings using the model
            embeddings = model(inputs)
            
            # Get mining output for the embeddings
            mined_indices = mining_func(embeddings, targets)
            
            # Calculate loss
            loss = loss_func(embeddings, targets, mined_indices)

            total_loss += loss.item()
            total_batches += 1

    # Compute the average validation loss
    average_loss = total_loss / total_batches if total_batches > 0 else 0
    return average_loss



import torch.nn as nn

class EEGNet7(nn.Module):
    def __init__(self):
        super(EEGNet7, self).__init__()
        # InstanceNorm1d normalizes each channel across its 500 values for each sample
        self.norm = nn.InstanceNorm1d(93, affine=False)  # Normalizes across the 500 values in each of the 93 channels
        self.act = nn.ReLU()  # or nn.SELU() , ReLU
        self.act2 = nn.Tanh()
        self.conv1 = nn.Conv2d(1, 256, (1, 4), padding='same')
        self.pool1 = nn.MaxPool2d((1, 2))
        self.conv2 = nn.Conv2d(256, 192, (4, 1), padding='same')
        self.pool2 = nn.MaxPool2d((2, 1))
        self.conv3 = nn.Conv2d(192, 128, (1, 4), padding='same')
        self.pool3 = nn.MaxPool2d((1, 2))
        self.conv4 = nn.Conv2d(128, 96, (4, 1), padding='same')
        self.pool4 = nn.MaxPool2d((2, 1))
        self.conv5 = nn.Conv2d(96, 64, (1, 4), padding='same')
        self.pool5 = nn.MaxPool2d((1, 2))
        self.conv6 = nn.Conv2d(64, 32, (4, 1), padding='same')
        self.pool6 = nn.MaxPool2d((2, 1))
        self.conv7 = nn.Conv2d(32, 16, (1, 2), padding='same')
        self.conv8 = nn.Conv2d(16, 2, (2, 1), padding='same')
        self.flatten = nn.Flatten()
        self.embedding = nn.Linear(1364, 128)  # Embedding layer

    def forward(self, x):
        #if x.dim() == 3:
            #x = x.unsqueeze(1)  # Adding a channel dimension if input is 3D
        x = self.norm(x)  # Apply InstanceNorm along the channel dimension (squeeze to 1D first)
        x = x.unsqueeze(1)  # Reshape to 4D again for Conv2d
        x = self.act(self.conv1(x))
        x = self.pool1(x)
        x = self.act(self.conv2(x))
        x = self.pool2(x)
        x = self.act(self.conv3(x))
        x = self.pool3(x)
        x = self.act(self.conv4(x))
        x = self.pool4(x)
        x = self.act(self.conv5(x))
        x = self.pool5(x)
        x = self.act(self.conv6(x))
        x = self.pool6(x)
        x = self.act(self.conv7(x))
        x = self.act(self.conv8(x))
        x = self.flatten(x)
        x = self.embedding(x)
        return x




from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()

def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch, test_loader):
    model.train()
    total_batches = len(train_loader)
    print(f"Total number of batches in train_loader: {total_batches}")

    for batch_idx, (data, labels, sessions) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()

        # Use mixed precision training
        with autocast():
            embeddings = model(data)
            indices_tuple = mining_func(embeddings, labels)
            loss = loss_func(embeddings, labels, indices_tuple)

        # Scales the loss, and backward pass
        scaler.scale(loss).backward()

        # Unscales gradients and optimizer step
        scaler.step(optimizer)
        scaler.update()

        if batch_idx % 100 == 0:
            print(
                "Epoch {} Iteration {}: Loss = {}, Number of mined P N = {} {}".format(
                    epoch, batch_idx, loss.item(), mining_func.num_pos_pairs, mining_func.num_neg_pairs  
                )
            )
    val_loss = validate(model, test_loader, loss_func, mining_func, device)
    print("Validation loss: ", val_loss)








device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("device", device)
trunk = EEGNet7().to(device)
summary(trunk, (93, sample_len)) 

loss_func = losses.SupConLoss(temperature=0.1).to(device)
#loss_func = losses.LiftedStructureLoss().to(device)

#sampler = samplers.MPerClassSampler(unique_ids, m=1, batch_size=batch_s)
sampler = samplers.MPerClassSampler(train_dataset.targets, m=4, batch_size=batch_s)


train_loader = DataLoader(train_dataset, batch_size=batch_s, sampler=sampler)
trunk_optimizer = torch.optim.Adam(trunk.parameters(), lr=0.0001)
#mining_func = miners.PairMarginMiner(pos_margin=0.20, neg_margin=0.8)
mining_func = miners.MultiSimilarityMiner(epsilon=0.1)

sampler2 = samplers.MPerClassSampler(valid_dataset.targets, m=4, batch_size=32)
valid_loader = DataLoader(valid_dataset, batch_size=batch_s, sampler=sampler2)


for epoch in range(1, num_epochs + 1):
    train(trunk, loss_func, mining_func, device, train_loader, trunk_optimizer, epoch,valid_loader)
    #test(trunk, loss_func, mining_func, device, test_loader)





torch.save(trunk.state_dict(), './model/SupConLoss128_m4_e99_unconnected.pth')

Number of unique subjects: 230
Number of unique subjects: 15
device cuda


/home/fallahi/anaconda3/envs/oml/lib/python3.9/site-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv2d(input, weight, bias, self.stride,


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
    InstanceNorm1d-1              [-1, 93, 500]               0
            Conv2d-2         [-1, 256, 93, 500]           1,280
              ReLU-3         [-1, 256, 93, 500]               0
         MaxPool2d-4         [-1, 256, 93, 250]               0
            Conv2d-5         [-1, 192, 93, 250]         196,800
              ReLU-6         [-1, 192, 93, 250]               0
         MaxPool2d-7         [-1, 192, 46, 250]               0
            Conv2d-8         [-1, 128, 46, 250]          98,432
              ReLU-9         [-1, 128, 46, 250]               0
        MaxPool2d-10         [-1, 128, 46, 125]               0
           Conv2d-11          [-1, 96, 46, 125]          49,248
             ReLU-12          [-1, 96, 46, 125]               0
        MaxPool2d-13          [-1, 96, 23, 125]               0
           Conv2d-14          [-1, 64, 

In [4]:
device

device(type='cuda')